# World Coordinate System (WCS)

#### What is a WCS?
The mapping from pixel coordinates to some physical coordinates (spatial, spectral, time, etc....)

The term WCS is often used to refer specifically to the most widely used [FITS WCS Standard](https://fits.gsfc.nasa.gov/fits_wcs.html) and its implementation.

## astropy.wcs

Implements the FITS WCS standard and some commonly used distortion conventions. The core standard is a wrapper around Mark Calabretta's [wcslib C library](https://www.atnf.csiro.au/people/mcalabre/WCS/index.html).

This tutorial will show how to create a WCS object from a FITS file and how to use it to transform coordinates.

In [ ]:
import os
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
from astropy.utils.data import get_pkg_data_filename  # astropy utility function 
from astropy.io import fits  # the astropy I/O module
from astropy import wcs   # the astropy WCS subpackage

## Creating a WCS object from the header of a FITS file

Open a file with `astropy.fits` and look at it. (This file contains only the header and no data)

In [ ]:
sip_file_name = get_pkg_data_filename('data/sip.fits', package='astropy.wcs.tests')

sip_file = fits.open(sip_file_name)
sip_file.info()

Let's look at the header of the file and identify the WCS keywords:

In [ ]:
sip_file[0].header


The Primary WCS keywords are:

```
WCSAXES =                    2 / Number of coordinate axes                      
CRPIX1  =                128.0 / Pixel coordinate of reference point            
CRPIX2  =                128.0 / Pixel coordinate of reference point            
PC1_1   =    0.000249756880272 / Coordinate transformation matrix element       
PC1_2   =    0.000230177809744 / Coordinate transformation matrix element       
PC2_1   =    0.000230428519265 / Coordinate transformation matrix element       
PC2_2   =   -0.000249965770577 / Coordinate transformation matrix element       
CDELT1  =                    1 / [deg] Coordinate increment at reference point  
CDELT2  =                    1 / [deg] Coordinate increment at reference point  
CUNIT1  = 'deg'                / Units of coordinate increment and value        
CUNIT2  = 'deg'                / Units of coordinate increment and value        
CTYPE1  = 'RA---TAN-SIP'       / Right ascension, gnomonic projection           
CTYPE2  = 'DEC--TAN-SIP'       / Declination, gnomonic projection               
CRVAL1  =        202.482322805 / [deg] Coordinate value at reference point      
CRVAL2  =          47.17511893 / [deg] Coordinate value at reference point      
LONPOLE =                  180 / [deg] Native longitude of celestial pole       
LATPOLE =          47.17511893 / [deg] Native latitude of celestial pole        
RESTFRQ =                    0 / [Hz] Line rest frequency                       
RESTWAV =                    0 / [Hz] Line rest wavelength                      
CRDER1  =    4.02509762361E-05 / [deg] Random error in coordinate               
CRDER2  =    3.42746131953E-05 / [deg] Random error in coordinate               
RADESYS = 'ICRS'               / Equatorial coordinate system                   
EQUINOX =                 2000 / [yr] Equinox of equatorial coordinates 
```

This file contains a distortion model represented as a [Simple Imaging Polynomial (SIP) distortion](https://fits.gsfc.nasa.gov/registry/sip.html).

```
BP_0_1  =          -1.6588E-05                                                  
BP_0_2  =          -2.3424E-05                                                  
A_3_0   =          -1.4172E-07                                                  
B_3_0   =          -2.0249E-08                                                  
BP_3_0  =           2.0482E-08                                                  
B_1_2   =          -5.7813E-09                                                  
B_1_1   =          -2.4386E-05                                                  
B_2_1   =          -1.6583E-07                                                  
B_2_0   =           2.1197E-06                                                  
A_ORDER =                    3                                                  
B_0_3   =          -1.6168E-07                                                  
B_0_2   =             2.31E-05                                                  
BP_0_3  =            1.651E-07                                                  
B_ORDER =                    3                                                  
BP_ORDER=                    3                                                  
BP_1_2  =           3.8917E-09                                                  
AP_ORDER=                    3                                                  
AP_3_0  =           1.4492E-07                                                  
A_1_1   =           2.1886E-05                                                  
BP_2_0  =           -2.151E-06                                                  
A_1_2   =          -1.6847E-07                                                  
AP_2_1  =            6.709E-09                                                  
AP_2_0  =           2.4146E-05                                                  
A_0_2   =           2.9656E-06                                                  
A_0_3   =           3.7746E-09                                                  
BP_1_1  =           2.4753E-05                                                  
BP_1_0  =          -2.6783E-06                                                  
A_2_0   =          -2.3863E-05                                                  
A_2_1   =           -8.561E-09                                                  
AP_1_0  =          -1.4897E-05                                                  
AP_1_1  =           -2.225E-05                                                  
AP_1_2  =           1.7195E-07                                                  
BP_2_1  =              1.7E-07                                                  
AP_0_1  =          -6.4275E-07                                                  
AP_0_3  =           -3.582E-09                                                  
AP_0_2  =          -2.9425E-06      
```

To create a WCS object pass the header with the WCS keywords to `astropy.wcs.WCS`. In this case it is the primary header. A warning is issued due to the lack of image data. 

In [ ]:
w = wcs.WCS(sip_file[0].header)

In [ ]:
print(w)

## Transforming between pixel coordinates and sky coordinates

To perform the transformation from detector to sky, including distortions, pass x and y and an 'origin'. The third argument, 'origin', indicates whether the coordinates are 1-based (like FITS), or 0-based (like python).

The inputs can be numbers, numpy arrays or array like objects.

In [ ]:
ra, dec = w.all_pix2world([1, 100], [2, 200], 1)
print(ra, dec)

Perfom the inverse transformation - from sky to detector coordinates.

If analytical inverse is not available (often the case in the presence of distortion), then an iterative inverse is performed.

In [ ]:
print(w.all_world2pix(ra, dec, 1))

In some cases it is useful to omit the distortion and perform the core WCS transforms only:

In [ ]:
ra, dec = w.wcs_pix2world([1, 100], [2, 200], 1)
print(ra, dec)

In [ ]:
w.wcs_world2pix(ra, dec, 1)

## Common WCS API (aka APE 14)

There are other implementations of a World Coordinate System. To unify the experience for the user, a [common API](https://zenodo.org/records/1188875) was created.

In [ ]:
sky = w.pixel_to_world([0, 99], [1, 199])  # Note that this assumes the coordinates are 0-based
print(sky)

In [ ]:
w.world_to_pixel(sky)

In [ ]:
sky_values = w.pixel_to_world_values([0, 99], [1, 199]) 

In [ ]:
w.world_to_pixel_values(sky_values)

## Creating a WCS programatically

A WCS object can be created programatically. Here is a concise example with 1 arcsecond pixels that is aligned with "North up, East to the left".

In [ ]:
my_wcs = wcs.WCS(naxis=2)
my_wcs.wcs.ctype = ['RA---TAN', 'DEC--TAN']
my_wcs.wcs.crpix = [512, 512]
my_wcs.wcs.crval = [70., 20.]
my_wcs.wcs.cdelt = [-1/3600, 1/3600]
my_wcs.array_shape = [1024, 1024] # NAXIS2, NAXIS1
my_wcs

## Converting a WCS into an astropy.io.fits.Header instance

The WCS object can be changed and the new WCS can be written out as a new header.

By default only the primary WCS keywords are written out to the header. Pass a keyword `relax=True` to write out the SIP distortion.

In [ ]:
# The original WCS
w.printwcs()

In [ ]:
w.wcs.crpix = [200, 200]

# Calling *to_header()* without arguments writes
# out the standard WCS keywords.
w.to_header()

In [ ]:
# Passing *relax=True* writes out the SIP coefficients
# and all other distortions.
w.to_header(relax=True)

## Generate a WCS object for an observation from the 2-m Rozhen telescope

In [ ]:
ls

In [ ]:
wr2 = wcs.WCS('rozhen2m_2.fits')
print(wr2)

Clearly there is no WCS information in the headers. 

Let's try [astrometry.net](https://nova.astrometry.net

I already uploaded the two images to astrometry.net and downloaded the new images with added pointing information and WCS keywords.
The links to the two uploads are here

https://nova.astrometry.net/user_images/8939748#annotated

https://nova.astrometry.net/user_images/8943580#annotated


The new files are called `new_rozhen2m_1.fits` and `new_rozhen2m_2.fits`

In [ ]:
fr1 = fits.open('new_rozhen2m_1.fits')
fr2 = fits.open('new_rozhen2m_2.fits')

In [ ]:
plt.figure(figsize=(10, 8))
plt.gray()
ax1 = plt.subplot(1,2,1)
ax1.imshow(fr1[0].data, origin='lower', vmin=1244, vmax=1390)
ax2 = plt.subplot(1,2,2)
ax2.imshow(fr2[0].data, origin='lower', vmin=1244, vmax=1390)


In [ ]:
from reproject.mosaicking import find_optimal_celestial_wcs

wcs_out, shape_out = find_optimal_celestial_wcs([fr1, fr2])
print(wcs_out)

In [ ]:
from reproject import reproject_interp
from reproject.mosaicking import reproject_and_coadd

array, footprint = reproject_and_coadd([fr1, fr2],
                                       wcs_out,
                                       shape_out=shape_out,
                                       reproject_function=reproject_interp)

In [ ]:
plt.figure(figsize=(10, 8))
plt.gray()
ax = plt.subplot(projection=wcs_out)
plt.grid(color='white', ls='solid')
plt.xlabel('Right Ascension')
plt.ylabel('Declination')

plt.imshow(array, origin='lower', vmin=1244, vmax=1390)

## Exercise: Refine a WCS using a list of detections and a reference catalog

Refine a WCS for a science image exposure from the Zwicky Transient Facility from these ingredients:
* An initial header
* A detection list cut at 17th magnitude, in file `data/ztf_detections_17thmag.csv`
* A reference catalog with coordinates and magnitudes from Gaia cut at 17 Gaia G magnitude, in `data/Gaia-gaia_dr2_source-ztf-20190606224213_000667_zr.csv`

The exercise makes use of `astropy.wcs`, `astropy.coordinates` and the projection capabilities of WCSAxes.

1. Read in the detection list and the reference catalog with `astropy.table.Table.read`
2. Calculate starting RAs and Decs for the detection list using the initial WCS
3. Create SkyCoord instances for the initial detection coordinates and the Gaia coordinates
4. Plot the detection list and the Gaia list in a scatter plot
5. Match the detection list and the Gaia list
6. Refine the WCS using the `fit_wcs_from_points` function from `astropy.wcs.utils`

Import everything we'll need for the exercise.

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.wcs import WCS
from astropy.wcs.utils import fit_wcs_from_points
import astropy.units as u

%matplotlib inline

Create the initial WCS programatically.

In [ ]:
initial_wcs = WCS(naxis=2)
initial_wcs.wcs.ctype = ['RA---TAN', 'DEC--TAN']  
initial_wcs.wcs.crval = [149.07662386535503, 33.32164150821777]  
initial_wcs.wcs.crpix = [-3305.678, -7136.481]
initial_wcs.wcs.cd = [[-0.0002817188, -1.554e-07],
                      [-1.998e-07, -0.0002819204]]  
initial_wcs.array_shape = [3080, 3072] # NAXIS2, NAXIS1

In [ ]:
initial_wcs

### 1. Read in the detection list and the reference catalog

Read in the detections and the reference catalog using `astropy.table.Table` with `format='csv'`.
The detections table is in `'data/ztf_detections_17thmag.csv'` and the reference catalog is `'data/Gaia-gaia_dr2_source-ztf-20190606224213_000667_zr.csv'`

In [ ]:
#detections = Table.read(...)

In [ ]:
#ref_catalog = Table.read(...)

### 2. Calculate starting RAs and Decs for the detection list using the initial WCS

Use the `initial_wcs.all_pix2world` function to calculate starting RA and Dec from the `detections['xpos']` and `detections['ypos']` columns. The pixel positions use the FITS numbering convention.

In [ ]:
#initial_ra, initial_dec = initial_wcs.all_pix2world(...)

### 3. Create SkyCoord instances for the initial detection coordinates and the Gaia coordinates

In [ ]:
#initial_coords = SkyCoord(...)

In [ ]:
#gaia_coords = SkyCoord(...)

### 4. Plot the detection list and the Gaia list in a scatter plot

Use `projection=initial_wcs` to make a scatter plot using `gaia_coords` and `initial_coords`. The open circles are sized according to magnitude.

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = plt.subplot(projection=initial_wcs)

# Uncomment this block to plot the Gaia coordinates
#ax.scatter(gaia_coords.ra, 
#           gaia_coords.dec,  c=None, marker='o',
#           s=20*(18 - ref_catalog['phot_g_mean_mag']),
#           facecolors='none', edgecolors='green',
#           transform=ax.get_transform('world'))

# Repeat for `initial_coords` but use `edgecolors='blue'

### 5. Match the detection list and the Gaia list

Use the `initial_coords.search_around_sky` method with a 15 arcsecond radius.

In [ ]:
#idxgaia, idxdet, d2d, d3d = initial_coords.search_around_sky(...)

In [ ]:
#gaia_matched = gaia_coords[idxgaia]
#detections_xpos_matched = detections['xpos'][idxdet]
#detections_ypos_matched = detections['ypos'][idxdet]
#print(len(gaia_matched), len(detections_xpos_matched), len(detections_ypos_matched))

### 6. Refine the WCS using the `fit_wcs_from_points` function

Look at the help for `fit_wcs_from_points` and use it to fit a new WCS. Use `sip_degree=3` to fit 3rd-order polynomial distortion.

Optionally, calculate new RAs and Decs for the matched pixel coordinates, and make another scatter plot.

In [ ]:
fit_wcs_from_points?

In [ ]:
#fitted_wcs = fit_wcs_from_points(...)

In [ ]:
#fitted_wcs

Examine the SIP distortion coefficients

In [ ]:
#fitted_wcs.sip.a

In [ ]:
#fitted_wcs.sip.b